# **Colección y procesamiento de los datos**
Autor: Christian Lara

Repo: [GitLab](https://git.uclv.edu.cu/clara/ampeptides/)


# Instalando Conda

In [89]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2024-01-13 06:39:24--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  53.4MB/s    in 1.5s    

2024-01-13 06:39:25 (53.4 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=

# **Descargar e Instalar biblioteca Pfeature**

In [90]:
! wget https://github.com/raghavagps/Pfeature/raw/master/PyLib/Pfeature.zip
! unzip Pfeature.zip
%cd Pfeature
! python setup.py install
%cd ..

--2024-01-13 06:40:21--  https://github.com/raghavagps/Pfeature/raw/master/PyLib/Pfeature.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/raghavagps/Pfeature/master/PyLib/Pfeature.zip [following]
--2024-01-13 06:40:21--  https://raw.githubusercontent.com/raghavagps/Pfeature/master/PyLib/Pfeature.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519486 (507K) [application/zip]
Saving to: ‘Pfeature.zip’

Pfeature.zip        100%[===================>] 507.31K  --.-KB/s    in 0.04s   

2024-01-13 06:40:21 (12.4 MB/s) - ‘Pfeature.zip’ saved [519486/519486]

Archive:  Pfeature.zip
   creating: P

# **Instalando CD-HIT**

In [91]:
! conda install -c bioconda cd-hit -y

Solving environment: - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cd-hit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.12.12 |       h06a4308_0         126 KB
    cd-hit-4.8.1               |       hdbcaa40_0         221 KB  bioconda
    certifi-2020.6.20          |     pyhd3eb1b0_3         155 KB
    openssl-1.1.1w             |       h7f8727e_0         3.7 MB
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following NEW packages will be INSTALLED:

  cd-hit             bioconda/linux-64::cd-hit-4.8.1-hdbcaa40_0

The following packages wi

# **Cargar Dataset de Péptidos**

In [ ]:
! wget https://github.com/chrislara01/AMP/blob/master/TR_starPep_AMP.fasta
! wget https://github.com/chrislara01/AMP/blob/master/EX_starPep_AMP.fasta
! wget https://github.com/chrislara01/AMP/blob/master/TS_starPep_AMP.fasta

--2024-01-13 05:48:34--  https://github.com/chrislara01/AMP/blob/master/TR_starPep_AMP.fasta
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4260 (4.2K) [text/plain]
Saving to: ‘TR_starPep_AMP.fasta’

TR_starPep_AMP.fast 100%[===================>]   4.16K  --.-KB/s    in 0.005s  

2024-01-13 05:48:34 (859 KB/s) - ‘TR_starPep_AMP.fasta’ saved [4260/4260]



# **Eliminando secuencias redundantes utilizando CD-HIT**

In [95]:
! cd-hit -i EX_starPep_AMP.fasta -o EX_starPep_AMP_cdhit.txt -c 0.99

Program: CD-HIT, V4.8.1, Mar 01 2019, 14:14:47
Command: cd-hit -i EX_starPep_AMP.fasta -o
         EX_starPep_AMP_cdhit.txt -c 0.99

Started: Sat Jan 13 06:41:47 2024
                            Output                              
----------------------------------------------------------------
total seq: 15318
longest and shortest : 100 and 11
Total letters: 507196
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 2M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 79M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90119636

comparing sequences from          0  to      15318
..........    10000  finished       9567  clusters
.....
    15318  finished      14296  clusters

Approximated maximum memory consumption: 82M
writing new database
writing clustering information
program completed !

Total CPU time 0.29


In [96]:
! cd-hit -i TR_starPep_AMP.fasta -o TR_starPep_AMP_cdhit.txt -c 0.99

Program: CD-HIT, V4.8.1, Mar 01 2019, 14:14:47
Command: cd-hit -i TR_starPep_AMP.fasta -o
         TR_starPep_AMP_cdhit.txt -c 0.99

Started: Sat Jan 13 06:42:24 2024
                            Output                              
----------------------------------------------------------------
Discarding invalid sequence or sequence without identifier and description!

{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":"Coleccion y procesamiento.ipynb","path":"Coleccion y procesamiento.ipynb","contentType":"file"},{"name":"Data Collection.ipynb","path":"Data Collection.ipynb","contentType":"file"},{"name":"TR_starPep_AMP.fasta","path":"TR_starPep_AMP.fasta","contentType":"file"}],"totalCount":3}},"fileTreeProcessingTime":1.759437,"foldersToFetch":[],"reducedMotionEnabled":null,"repo":{"id":742319498,"defaultBranch":"master","name":"AMP","ownerLogin":"chrislara01","currentUserCanPush":false,"isFork":false,"isEmpty":false,"createdAt":"2024-01-12T08:02:43.000Z","ow

In [97]:
! cd-hit -i TS_starPep_AMP.fasta -o TS_starPep_AMP_cdhit.txt -c 0.99

Program: CD-HIT, V4.8.1, Mar 01 2019, 14:14:47
Command: cd-hit -i TS_starPep_AMP.fasta -o
         TS_starPep_AMP_cdhit.txt -c 0.99

Started: Sat Jan 13 06:42:28 2024
                            Output                              
----------------------------------------------------------------
total seq: 4452
longest and shortest : 100 and 11
Total letters: 157377
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90401149

comparing sequences from          0  to       4452
....
     4452  finished       4244  clusters

Approximated maximum memory consumption: 77M
writing new database
writing clustering information
program completed !

Total CPU time 0.15


# **Preprocesando los archivos fasta**

In [69]:
import pandas as pd
from Pfeature.pfeature import *

In [64]:
def split_np(file):
    """Divide archivos txt de tipo fasta en sets positivos y negativos"""
    with open (file, "r") as fasta:
        pos = ''
        neg = ''
        fastaread = fasta.read()
        array = fastaread.split('>')[1:]
        for line in array:
            if 'nonAMP' in line:
                neg += ('>' + line)
            else:
                pos += ('>' + line)
    pos = pos.removesuffix('\n')
    neg = neg.removesuffix('\n')
    with open(file.replace('.txt', '_pos.txt'), "w") as test:
        test.write(pos)
    with open(file.replace('.txt', '_neg.txt'), "w") as test:
        test.write(neg)
    return array

In [65]:
array = split_np('TS_starPep_AMP_cdhit.txt')

In [73]:
#Definiendo funciones para el calculo de features de los datasets

def aac(input):
  a = input.rstrip('txt')
  output = a + 'aac.csv'
  df_out = aac_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

def pcp(input):
  a = input.rstrip('txt')
  output = a + 'pcp.csv'
  df_out = pcp_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

In [79]:
# Metodo para concatenar los features calculados de los datasets positivo y negativo

def feature_calc(po, ne, feature_name):
  # Calculate feature
  po_feature = feature_name(po)
  ne_feature = feature_name(ne)
  # Create class labels
  po_class = pd.Series(['positive' for i in range(len(po_feature))])
  ne_class = pd.Series(['negative' for i in range(len(ne_feature))])
  # Combine po and ne
  po_ne_class = pd.concat([po_class, ne_class], axis=0)
  po_ne_class.name = 'class'
  po_ne_feature = pd.concat([po_feature, ne_feature], axis=0)
  # Combine feature and class
  df = pd.concat([po_ne_feature, po_ne_class], axis=1)
  return df

In [ ]:
# Encapsulamiento de la division y concatenacion en el calculo de features
# en archivos txt

def fastaFeatureCalc(input, feature):
    split_np(input)
    pos = input.replace('.txt', '_pos.txt')
    neg = input.replace('.txt', '_neg.txt')
    return feature_calc(pos, neg, feature)